The Microsoft Scalable Noisy Speech Dataset (MS-SNSD) is a noisy speech dataset that can scale to arbitrary sizes depending on the number of speakers, noise types, and Speech to Noise Ratio (SNR) levels desired.

a noisy speech dataset (MS-SNSD) that can scale to arbitrary sizes depending on the number of speakers, noise types, and Speech to Noise Ratio (SNR) levels desired

n data augmentation 
Source: https://github.com/microsoft/MS-SNSD


In [32]:
import os
cwd = os.getcwd()
cwd

'/Users/michliu/Documents/HSBC - Data scientist/audio/cantonese2'

In [33]:
os. chdir('/Users/michliu/Documents/HSBC - Data scientist/audio/cantonese2/clean_train') 

In [34]:
clean_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), 'clean_train')
print('clean_dir', clean_dir)

clean_dir /Users/michliu/Documents/HSBC - Data scientist/audio/cantonese2/clean_train/clean_train


In [35]:
# Convert file to wav. format

from os import path
from pydub import AudioSegment
# files                                                                         
src = "Cantonese1.mov"
dst = "Cantonese1.wav"
#cleanfilenames = glob.glob(os.path.join(src, ".mov"))
# convert mov to wav                                                            
sound = AudioSegment.from_file(src, format="mov")
sound = sound.set_channels(1)
sound.export(dst, format="wav")

<_io.BufferedRandom name='Cantonese1.wav'>

In [36]:
src = "Cantonese2.mov"
dst = "Cantonese2.wav"
#cleanfilenames = glob.glob(os.path.join(src, ".mov"))
# convert mov to wav                                                            
sound = AudioSegment.from_file(src, format="mov")
sound = sound.set_channels(1)
sound.export(dst, format="wav")
src = "Cantonese3.mov"
dst = "Cantonese3.wav"
#cleanfilenames = glob.glob(os.path.join(src, ".mov"))
# convert mov to wav                                                            
sound = AudioSegment.from_file(src, format="mov")
sound = sound.set_channels(1)
sound.export(dst, format="wav")
src = "Cantonese4.mov"
dst = "Cantonese4.wav"
#cleanfilenames = glob.glob(os.path.join(src, ".mov"))
# convert mov to wav                                                            
sound = AudioSegment.from_file(src, format="mov")
sound = sound.set_channels(1)
sound.export(dst, format="wav")
src = "Cantonese5.mov"
dst = "Cantonese5.wav"
#cleanfilenames = glob.glob(os.path.join(src, ".mov"))
# convert mov to wav                                                            
sound = AudioSegment.from_file(src, format="mov")
sound = sound.set_channels(1)
sound.export(dst, format="wav")

<_io.BufferedRandom name='Cantonese5.wav'>

In [37]:
os. chdir('/Users/michliu/Documents/HSBC - Data scientist/audio/cantonese2') 

In [38]:
import soundfile as sf
import os
import numpy as np
import glob
import sounddevice as sd

# Function to read audio
def audioread(path, norm = True, start=0, stop=None):
    path = os.path.abspath(path)
    if not os.path.exists(path):
        raise ValueError("[{}] does not exist!".format(path))
    try:
        x, sr = sf.read(path, start=start, stop=stop)
    except RuntimeError:  # fix for sph pcm-embedded shortened v2
        print('WARNING: Audio type not supported')

    if len(x.shape) == 1:  # mono
        if norm:
            rms = (x ** 2).mean() ** 0.5
            scalar = 10 ** (-25 / 20) / (rms)
            x = x * scalar
        return x, sr
    else:  # multi-channel
        x = x.T
        x = x.sum(axis=0)/x.shape[0]
        if norm:
            rms = (x ** 2).mean() ** 0.5
            scalar = 10 ** (-25 / 20) / (rms)
            x = x * scalar
        return x, sr
    
# Funtion to write audio    
def audiowrite(data, fs, destpath, norm=False):
    if norm:
        rms = (data ** 2).mean() ** 0.5
        scalar = 10 ** (-25 / 10) / (rms+eps)
        data = data * scalar
        if max(abs(data))>=1:
            data = data/max(abs(data), eps)
    
    destpath = os.path.abspath(destpath)
    destdir = os.path.dirname(destpath)
    
    if not os.path.exists(destdir):
        os.makedirs(destdir)
    
    sf.write(destpath, data, fs)
    return

# Function to mix clean speech and noise at various signal to noise ratio (SNR) levels
def snr_mixer(clean, noise, snr):
    # Normalizing to -25 dB FS
    rmsclean = (clean**2).mean()**0.5
    scalarclean = 10 ** (-25 / 20) / rmsclean
    clean = clean * scalarclean
    rmsclean = (clean**2).mean()**0.5

    rmsnoise = (noise**2).mean()**0.5
    scalarnoise = 10 ** (-25 / 20) /rmsnoise
    noise = noise * scalarnoise
    rmsnoise = (noise**2).mean()**0.5
    
    # Set the noise level for a given SNR
    noisescalar = np.sqrt(rmsclean / (10**(snr/20)) / rmsnoise)
    noisenewlevel = noise * noisescalar
    noisyspeech = clean + noisenewlevel
    return clean, noisenewlevel, noisyspeech
        
    


In [79]:
# Configuration for generating Noisy Speech Dataset

# - sampling_rate: Specify the sampling rate. Default is 16 kHz
# - audioformat: default is .wav
# - audio_length: Max Length of each audio clip (noisy and clean speech) in seconds that will be generated by augmenting utterances. 
# - silence_length: Duration of silence introduced between clean speech utterances.
# - total_hours: Total number of hours of data required. Units are in hours. 
# - snr_lower: Lower bound for SNR required (default: 0 dB)
# - snr_upper: Upper bound for SNR required (default: 40 dB)
# - total_snrlevels: Number of SNR levels required (default: 5, which means there are 5 levels between snr_lower and snr_upper)
# - noise_dir: Default is None. But specify the noise directory path if noise files are not in the source directory
# - Speech_dir: Default is None. But specify the speech directory path if speech files are not in the source directory
# - noise_types_excluded: Noise files starting with the following tags to be excluded in the noise list. Example: noise_types_excluded: Babble, AirConditioner
#                         Specify 'None' if no noise files to be excluded.

cfg={
'sampling_rate': 44100,
'audioformat': '*.wav',
'audio_length': 60,
'silence_length': 0.2,
'total_hours': 1 ,
'snr_lower': 0,
'snr_upper': 40,
'total_snrlevels': 5,  

'noise_dir': None,
'speech_dir': None,
'noise_types_excluded': 'SqueakyChair',
}

In [80]:
snr_lower = int(cfg['snr_lower'])
snr_upper = int(cfg['snr_upper'])
total_snrlevels = int(cfg['total_snrlevels'])

## create noisy training file

In [99]:
clean_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), 'clean_train')
print('clean_dir', clean_dir)
noise_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), 'noise_train')
print('noise_dir', noise_dir)

clean_dir /Users/michliu/Documents/HSBC - Data scientist/audio/cantonese2/clean_train
noise_dir /Users/michliu/Documents/HSBC - Data scientist/audio/cantonese2/noise_train


In [100]:
fs = float(cfg["sampling_rate"])
audioformat = cfg["audioformat"]
total_hours = float(cfg["total_hours"])
audio_length = float(cfg["audio_length"])
silence_length = float(cfg["silence_length"])
noisyspeech_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), 'NoisySpeech_training')
if not os.path.exists(noisyspeech_dir):
    os.makedirs(noisyspeech_dir)
clean_proc_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), 'CleanSpeech_training')
if not os.path.exists(clean_proc_dir):
    os.makedirs(clean_proc_dir)
noise_proc_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), 'Noise_training')
if not os.path.exists(noise_proc_dir):
    os.makedirs(noise_proc_dir)

In [101]:
total_secs = total_hours*60*60
total_samples = int(total_secs * fs)
audio_length = int(audio_length*fs)
SNR = np.linspace(snr_lower, snr_upper, total_snrlevels)
cleanfilenames = glob.glob(os.path.join(clean_dir, audioformat))
if cfg["noise_types_excluded"]==None:
    noisefilenames = glob.glob(os.path.join(noise_dir, audioformat))
else:
    filestoexclude = cfg["noise_types_excluded"].split(',')
    noisefilenames = glob.glob(os.path.join(noise_dir, audioformat))
    for i in range(len(filestoexclude)):
        noisefilenames = [fn for fn in noisefilenames if not os.path.basename(fn).startswith(filestoexclude[i])]
    

In [102]:

filecounter = 0
num_samples = 0
#fs2= 44100
    
while num_samples < total_samples:
    idx_s = np.random.randint(0, np.size(cleanfilenames))
    #print('cleanfilenames[idx_s]', cleanfilenames[idx_s])
    clean, fs = audioread(cleanfilenames[idx_s])
    
    if len(clean)<audio_length:
        clean = clean
       
        
    else:
            
        while len(clean)<=audio_length:
            idx_s = idx_s + 1
            if idx_s >= np.size(cleanfilenames)-1:
                idx_s = np.random.randint(0, np.size(cleanfilenames)) 
            newclean, fs = audioread(cleanfilenames[idx_s])
            cleanconcat = np.append(clean, np.zeros(int(fs*silence_length)))
            clean = np.append(cleanconcat, newclean)


    idx_n = np.random.randint(0, np.size(noisefilenames))
    noise, fs2 = audioread(noisefilenames[idx_n])
    idx2 = idx_n
    
    # Resample data
    new_rate = 44100
    number_of_samples = round(len(noise) * float(new_rate) / fs2)
    noise = sps.resample(noise, number_of_samples)
    
    
    
    
        
    if len(noise)>=len(clean):
        noise = noise[0:len(clean)]
        #noise_filename = noisefilenames[idx_n]
        
    else:
        
        while len(noise)<=len(clean):
            idx_n = idx_n + 1
            if idx_n >= np.size(noisefilenames)-1:
                idx_n = np.random.randint(0, np.size(noisefilenames))
            newnoise, fs2 = audioread(noisefilenames[idx_n])
            idx2 = idx_n
             # Resample data
            new_rate = 44100
            number_of_samples = round(len(noise) * float(new_rate) / fs2)
            noise = sps.resample(noise, number_of_samples)
            
            noiseconcat = np.append(noise, np.zeros(int(fs*silence_length)))
            noise = np.append(noiseconcat, newnoise)
            #noise_filename = noisefilenames[idx_n]
    #print((noise_filename.split("/")[-1].split('.')[0]))
    noise = noise[0:len(clean)]
    filecounter = filecounter + 1

        
    #print('filecounter', filecounter)
    
    

    for i in range(np.size(SNR)):
        clean_snr, noise_snr, noisy_snr = snr_mixer(clean=clean, noise=noise, snr=SNR[i])
        noise_filename = noisefilenames[idx2]
        print((noise_filename.split("/")[-1].split('.')[0]))
        
        noisyfilename = 'noisy_'+(cleanfilenames[idx_s].split("/")[-1].split('.')[0])+'_SNRdb_'+str(int(SNR[i]))+ '_'+ (noise_filename.split("/")[-1].split('.')[0]) +'.wav'
        cleanfilename = 'clnsp_'+(cleanfilenames[idx_s].split("/")[-1].split('.')[0])+'.wav'
        noisefilename = 'noisy_'+(cleanfilenames[idx_s].split("/")[-1].split('.')[0])+'_SNRdb_'+str(int(SNR[i]))+ '_'+ (noise_filename.split("/")[-1].split('.')[0]) +'.wav'
        noisypath = os.path.join(noisyspeech_dir, noisyfilename)
        cleanpath = os.path.join(clean_proc_dir, cleanfilename)
        noisepath = os.path.join(noise_proc_dir, noisefilename)
        print(noisyfilename )
        audiowrite(noisy_snr, fs, noisypath, norm=False)
        audiowrite(clean, fs, cleanpath, norm=False)
        audiowrite(noise_snr, fs, noisepath, norm=False)
        num_samples = num_samples + len(noisy_snr)
        
        #sd.play(clean, fs)
        #status = sd.wait() 

Typing_8
noisy_Cantonese3_SNRdb_0_Typing_8.wav
Typing_8
noisy_Cantonese3_SNRdb_10_Typing_8.wav
Typing_8
noisy_Cantonese3_SNRdb_20_Typing_8.wav
Typing_8
noisy_Cantonese3_SNRdb_30_Typing_8.wav
Typing_8
noisy_Cantonese3_SNRdb_40_Typing_8.wav
AirportAnnouncements_5
noisy_Cantonese2_SNRdb_0_AirportAnnouncements_5.wav
AirportAnnouncements_5
noisy_Cantonese2_SNRdb_10_AirportAnnouncements_5.wav
AirportAnnouncements_5
noisy_Cantonese2_SNRdb_20_AirportAnnouncements_5.wav
AirportAnnouncements_5
noisy_Cantonese2_SNRdb_30_AirportAnnouncements_5.wav
AirportAnnouncements_5
noisy_Cantonese2_SNRdb_40_AirportAnnouncements_5.wav
CopyMachine_10
noisy_Cantonese3_SNRdb_0_CopyMachine_10.wav
CopyMachine_10
noisy_Cantonese3_SNRdb_10_CopyMachine_10.wav
CopyMachine_10
noisy_Cantonese3_SNRdb_20_CopyMachine_10.wav
CopyMachine_10
noisy_Cantonese3_SNRdb_30_CopyMachine_10.wav
CopyMachine_10
noisy_Cantonese3_SNRdb_40_CopyMachine_10.wav
Munching_9
noisy_Cantonese5_SNRdb_0_Munching_9.wav
Munching_9
noisy_Cantonese5_SNRd

In [ ]:
np.size(noisefilenames)